In [34]:
import psycopg
import credentials
import pandas as pd
from datetime import datetime
import numpy as np
import sys

In [35]:
# Load data
# opening the file in read mode
hhs_df = pd.read_csv("/Users/sunyining/stat650/614_Project/test_data.csv")

# Data wrangling
hhs_df.replace(-999999, np.nan)
hhs_df["collection_week"] = \
    hhs_df.collection_week.apply(
        lambda x: datetime.strptime(str(x), '%d/%m/%Y'))
hhs_df["geocoded_hospital_address"] = hhs_df.geocoded_hospital_address.apply(lambda x: str(x))
hhs_df["zip"] = hhs_df.zip.apply(lambda x: str(x))
hhs_df["city"] = hhs_df.city.apply(lambda x: str(x))
hhs_df["fips_code"] = hhs_df.fips_code.apply(lambda x: str(x))
hhs_df["state"] = hhs_df.state.apply(lambda x: str(x))
hhs_df["address"] = hhs_df.address.apply(lambda x: str(x))

hhs_df["hospital_pk"] = hhs_df.hospital_pk.apply(lambda x: str(x))
hhs_df["all_adult_hospital_beds_7_day_avg"] = hhs_df.all_adult_hospital_beds_7_day_avg.apply(lambda x: float(x))
hhs_df["all_pediatric_inpatient_beds_7_day_avg"] = hhs_df.all_pediatric_inpatient_beds_7_day_avg.apply(lambda x: float(x))
hhs_df["all_adult_hospital_inpatient_bed_occupied_7_day_coverage"] = hhs_df.all_adult_hospital_inpatient_bed_occupied_7_day_coverage.apply(lambda x: float(x))
hhs_df["all_pediatric_inpatient_bed_occupied_7_day_avg"] = hhs_df.all_pediatric_inpatient_bed_occupied_7_day_avg.apply(lambda x: float(x))
hhs_df["total_icu_beds_7_day_avg"] = hhs_df.total_icu_beds_7_day_avg.apply(lambda x: float(x))
hhs_df["icu_beds_used_7_day_avg"] = hhs_df.icu_beds_used_7_day_avg.apply(lambda x: float(x))
hhs_df["inpatient_beds_used_covid_7_day_avg"] = hhs_df.inpatient_beds_used_covid_7_day_avg.apply(lambda x: float(x))
hhs_df["staffed_icu_adult_patients_confirmed_covid_7_day_avg"] = hhs_df.staffed_icu_adult_patients_confirmed_covid_7_day_avg.apply(lambda x: float(x))


In [36]:
# Insert data into table
conn = psycopg.connect(
    host="sculptor.stat.cmu.edu", dbname="yinings",
    user="yinings", password="shai0Asho"
)

# Already existed data
d = pd.read_sql_query("SELECT geocoded_hospital_address FROM geographic_info", conn)
old_hospitals_address = set(d.geocoded_hospital_address.unique())
d = pd.read_sql_query("SELECT hospital_pk FROM hospital_capacity", conn)
old_hospitals_pk = set(d.hospital_pk.unique())

# Data to be checked
check_data = pd.DataFrame(columns = hhs_df.columns)
check_data.columns

# Initial count
num_rows_inserted = 0

cur = conn.cursor()

/Users/sunyining/opt/anaconda3/lib/python3.9/site-packages/pandas/io/sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(
/Users/sunyining/opt/anaconda3/lib/python3.9/site-packages/pandas/io/sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


In [37]:
# make a new transaction
with conn.transaction():
    for index, row in hhs_df.iterrows():
        try:
            # make a new SAVEPOINT -- like a save in a video game
            with conn.transaction():
                # perhaps a bunch of reformatting
                # and data manipulation goes here

                # Update geocoded_hospital_address
                if row['geocoded_hospital_address'] not in old_hospitals_address:
                    # now insert the data
                    cur.execute(
                        "INSERT INTO geographic_info (geocoded_hospital_address, zip, city, fips_code, state, address)"
                        "VALUES (%s, %s, %s, %s, %s, %s)",
                        (row['geocoded_hospital_address'], row['zip'],
                         row['city'], row['fips_code'], row['state'], row['address']))

                # else:
                #     # now insert the data
                #     cur.execute(
                #         "UPDATE geographic_info SET \
                #             geocoded_hospital_address = %s, \
                #                 zip = %s, city = %s, fips_code = %s, state = %s, address = %s"
                #                 "WHERE geocoded_hospital_address = %s",
                #         (row.geocoded_hospital_address, row.zip,
                #          row.city, row.fips_code, row.state, row.address,
                #          row.geocoded_hospital_address))

                # Update hospital_capacity

                if row['hospital_pk'] not in old_hospitals_pk:
                    # now insert the data
                    cur.execute(
                        "INSERT INTO hospital_capacity (hospital_pk, date, \
                            all_adult_hospital_beds_7_day_avg, \
                                all_pediatric_inpatient_beds_7_day_avg, \
                                all_adult_hospital_inpatient_bed_occupied_7_day_coverage, \
                                    all_pediatric_inpatient_bed_occupied_7_day_avg, \
                                        total_icu_beds_7_day_avg, \
                                        icu_beds_used_7_day_avg, \
                                            inpatient_beds_used_covid_7_day_avg, \
                                            staffed_icu_adult_patients_confirmed_covid_7_day_avg)"
                        "VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s)",
                        (row['hospital_pk'], 
                         row['collection_week'],
                         row['all_adult_hospital_beds_7_day_avg'],
                         row['all_pediatric_inpatient_beds_7_day_avg'],
                         row['all_adult_hospital_inpatient_bed_occupied_7_day_coverage'],
                         row['all_pediatric_inpatient_bed_occupied_7_day_avg'],
                         row['total_icu_beds_7_day_avg'],
                         row['icu_beds_used_7_day_avg'],
                         row['inpatient_beds_used_covid_7_day_avg'],
                         row['staffed_icu_adult_patients_confirmed_covid_7_day_avg']))
        except Exception:
            # if an exception/error happens in this block,
            # Postgres goes back to the last savepoint upon
            # exiting the `with` block
            print("insert failed")
            # add additional logging, error handling here
            # check_data.append(row)

        # else:
        #     # no exception happened, so we continue
        #     # without reverting the savepoint
        #     num_rows_inserted += 1

In [38]:
# now we commit the entire transaction
conn.commit()
conn.close()
check_data.to_csv("data_to_be_checked.csv")